In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os

In [ ]:
dataset , info = tfds.load(name='mnist', with_info=True, as_supervised=True)

train, test = dataset['train'], dataset['test']

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
strategy = tf.distribute.MirroredStrategy()

print(strategy.num_replicas_in_sync)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


1


In [ ]:
num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

print(num_train_examples, num_test_examples)

60000 10000


In [ ]:
BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [ ]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255
  return image, label

In [ ]:
train_dataset = train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test.map(scale).cache().batch(BATCH_SIZE)

In [ ]:
with strategy.scope():
  model = tf.keras.Sequential([
                                      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape = (28, 28, 1)),
                                      tf.keras.layers.MaxPooling2D(),
                                      tf.keras.layers.Flatten(),
                                      tf.keras.layers.Dense(64, activation='relu'),
                                      tf.keras.layers.Dense(10)
                                      
  ])

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


In [ ]:
model.fit(train_dataset, epochs=12)

Epoch 1/12
938/938 [==============================] - 35s 34ms/step - loss: 0.2076 - accuracy: 0.9396
Epoch 2/12
938/938 [==============================] - 26s 28ms/step - loss: 0.0677 - accuracy: 0.9805
Epoch 3/12
938/938 [==============================] - 26s 28ms/step - loss: 0.0472 - accuracy: 0.9863
Epoch 4/12
938/938 [==============================] - 26s 28ms/step - loss: 0.0353 - accuracy: 0.9887
Epoch 5/12
938/938 [==============================] - 25s 27ms/step - loss: 0.0265 - accuracy: 0.9916
Epoch 6/12
938/938 [==============================] - 25s 27ms/step - loss: 0.0208 - accuracy: 0.9936
Epoch 7/12
938/938 [==============================] - 25s 27ms/step - loss: 0.0152 - accuracy: 0.9954
Epoch 8/12
938/938 [==============================] - 25s 27ms/step - loss: 0.0123 - accuracy: 0.9961
Epoch 9/12
938/938 [==============================] - 25s 27ms/step - loss: 0.0097 - accuracy: 0.9972
Epoch 10/12
938/938 [==============================] - 25s 27ms/step - loss: 0.007